## RAC/DVR: step 1

In [1]:
turning_points=True
safe_data=False

In [2]:
import numpy as np
import sys
from scipy.optimize import root_scalar
import matplotlib.pyplot as plt
%matplotlib qt5
import pandas as pd
#
#  extend path by location of the dvr package
#
sys.path.append('../../Python_libs')
import dvr
import jolanta

In [3]:
amu_to_au=1822.888486192
au2cm=219474.63068
au2eV=27.211386027
Angs2Bohr=1.8897259886

In [4]:
#
# Jolanata-3D parameters a, b, c: (0.028, 1.0, 0.028)
#
#   CS-DVR:   
#      bound state:  -7.17051 eV
#      resonance (3.1729556 - 0.16085j) eV
#
jparam=(0.028, 1.0, 0.028)

In [5]:
#
#  compute DVR of T and V
#  then show the density of states
#  in a potential + energy-levels plot
#  the standard 3D-Jolanta is used (resonance at 1.75 -0.2i eV)
#
rmin=0
rmax=12      # grid from 0 to rmax
thresh = 8   # maximum energy for plot
ppB = 15     # grid points per Bohr

nGrid=int((rmax-rmin)*ppB)
rs = dvr.DVRGrid(rmin, rmax, nGrid)
Vs = jolanta.Jolanta_3D(rs, jparam)
Ts = dvr.KineticEnergy(1, rmin, rmax, nGrid)
[energy, wf] = dvr.DVRDiag2(nGrid, Ts, Vs, wf=True)

n_ene=0
for i in range(nGrid):
    print("%3d  %12.8f au = %12.5f eV" % (i+1, energy[i], energy[i]*au2eV))
    n_ene += 1
    if energy[i]*au2eV > thresh:
        break

# "DVR normalization", sum(wf[:,0]**2)
# this is correct for plotting

c=["orange", "blue"]
#h=float(xmax) / (nGrid+1.0)
scale=3*au2eV

plt.cla()
plt.plot(rs,Vs*au2eV, '-', color="black")
for i in range(n_ene):
    plt.plot(rs, scale*wf[:,i]**2+energy[i]*au2eV, '-', color=c[i%len(c)])
plt.ylim(-8, 1.5*thresh)
plt.xlabel('$r$ [Bohr]')
plt.ylabel('$E$ [eV]')
plt.show()

  1   -0.26351095 au =     -7.17050 eV
  2    0.11989697 au =      3.26256 eV
  3    0.28142119 au =      7.65786 eV
  4    0.52212147 au =     14.20765 eV


### Test a turning point search

Find the outer turning point of $V(\lambda)$ at $E_r$

In [13]:
def f(x, E, jparam):
    """  equation V(x) - E = 0  """
    return jolanta.Jolanta_3D(x, jparam) - E

def find_brackets(rs, vs):
    """  from V(r) on DVR grid get [r(V_min), r(V(max+)] """
    imin = np.argmin(vs)
    imax = np.argmax(vs[imin:])
    return [rs[imin], rs[imin+imax]]

args=(energy[1],jparam)
bracket=find_brackets(rs,Vs)
print(bracket)
print(energy[1])
sol=root_scalar(f, args=args, bracket=bracket)
print('Turning point at %f Bohr.' % (sol.root))

[2.187845303867403, 8.353591160220994]
0.1198969707674879
Turning point at 7.095117 Bohr.


## RAC by increasing $b$

The last energy needs to be about $7E_r \approx 22$eV

In [7]:
#
#  show the potential
#
a_ref, b_ref, c_ref = jparam
plt.cla()
for b_curr in [1.1, 1.3, 1.5, 1.7]:
    param = [a_ref, b_curr, c_ref]
    plt.plot(rs, jolanta.Jolanta_3D(rs, param)*au2eV)

plt.ylim(-30, 10)
plt.show()

In [14]:
a_ref, b_ref, c_ref = jparam

b_min=b_ref
b_max=2.5
nEs_keep=4    # how many energies are kept

n_b=101

bs=np.linspace(b_min, b_max, num=n_b, endpoint=True)

rtp_b=np.zeros(n_b)

run_data = np.zeros((n_b, nEs_keep+1))  # array used to collect all eta-run data
run_data[:,0]=bs

for l, b_curr in enumerate(bs):
    param = [a_ref, b_curr, c_ref]
    Vs = jolanta.Jolanta_3D(rs, param)
    energy = dvr.DVRDiag2(nGrid, Ts, Vs)
    run_data[l,1:] = au2eV*energy[0:nEs_keep]
    if turning_points:
        args=(energy[1],param)
        bracket = find_brackets(rs,Vs)
        sol=root_scalar(f, args=args, bracket=bracket)
        rtp_b[l]=sol.root
    print(l+1, end=" ")
    if (l+1)%10==0:
        print()

if turning_points:
    Ebs = run_data[:,1]
        
print(run_data[-1,:])

1 2 3 4 5 6 7 8 9 10 
11 12 13 14 15 16 17 18 19 20 
21 22 23 24 25 26 27 28 29 30 
31 32 33 34 35 36 37 38 39 40 
41 42 43 44 45 46 47 48 49 50 
51 52 53 54 55 56 57 58 59 60 
61 62 63 64 65 66 67 68 69 70 
71 72 73 74 75 76 77 78 79 80 
81 82 83 84 85 86 87 88 89 90 
91 92 93 94 95 96 97 98 99 100 
101 [  2.5        -40.14360549 -21.72431982  -7.33018023   1.92800864]


In [15]:
plt.cla()
for i in range(0, nEs_keep):
    plt.plot(bs, run_data[:,i+1], 'o-')
plt.ylim(-25,5)
plt.show()

In [16]:
if turning_points:
    plt.cla()
    plt.plot(Ebs, rtp_b, '.-', label="b-scaling")
    plt.legend()
    plt.show()  

In [17]:
if safe_data:
    cols = ['z']
    for i in range(nEs_keep):
        cols.append('E'+str(i+1))
    df = pd.DataFrame(run_data, columns=cols)
    df.to_csv('rac_DVR_3D_b-scale_rmax_12.csv', index=False)
    df.head(5)

## RAC with Coulomb potential

In [34]:
def coulomb(r, lbd=1.0):
    """ attractive Coulomb potential with strength lbd = lamda """
    return -lbd/r

def f(x, E, jparam, l_curr):
    return jolanta.Jolanta_3D(x, jparam) + coulomb(x, lbd=l_curr) - E

In [35]:
#
#  show the potential
#
plt.cla()
for l_curr in [0, 0.5, 1.0, 1.5, 2.0]:
    vs = jolanta.Jolanta_3D(rs, jparam)+coulomb(rs, lbd=l_curr)
    plt.plot(rs, vs*au2eV)
    if turning_points:
        print(find_brackets(rs,vs))
        
#plt.xlim(0,15)
plt.ylim(-30, 10)
plt.show()

[2.187845303867403, 8.353591160220994]
[1.856353591160221, 8.552486187845304]
[1.5248618784530386, 8.817679558011049]
[1.1933701657458564, 9.082872928176796]
[0.9281767955801106, 9.348066298342541]


In [38]:
l_min=0.0
l_max=2.4
nEs_keep=4    # how many energies are kept

npts=101

ls=np.linspace(l_min, l_max, num=npts, endpoint=True)

rtp_C=np.zeros(npts)


run_data = np.zeros((npts, nEs_keep+1))  # array used to collect all eta-run data
run_data[:,0]=ls

VJs = jolanta.Jolanta_3D(rs, jparam)
Ws = coulomb(rs, lbd=1.0)

for j, l_curr in enumerate(ls):
    Vs = VJs + l_curr*Ws
    energy = dvr.DVRDiag2(nGrid, Ts, Vs)
    run_data[j,1:] = au2eV*energy[0:nEs_keep]
    if turning_points:
        args=(energy[1],jparam,l_curr)
        bracket = find_brackets(rs,Vs)
        sol=root_scalar(f, args=args, bracket=bracket)
        rtp_C[j]=sol.root
    print(j+1, end=" ")
    if (j+1)%10==0:
        print()

if turning_points:
    ECs = run_data[:,1]
print(run_data[-1,:])

1 2 3 4 5 6 7 8 9 10 
11 12 13 14 15 16 17 18 19 20 
21 22 23 24 25 26 27 28 29 30 
31 32 33 34 35 36 37 38 39 40 
41 42 43 44 45 46 47 48 49 50 
51 52 53 54 55 56 57 58 59 60 
61 62 63 64 65 66 67 68 69 70 
71 72 73 74 75 76 77 78 79 80 
81 82 83 84 85 86 87 88 89 90 
91 92 93 94 95 96 97 98 99 100 
101 [  2.4        -41.37210293 -19.35877462  -6.10967756   0.17117461]


In [42]:
if turning_points:
    plt.cla()
    plt.plot(Ebs, rtp_b, '.-', label="b-scaling")
    plt.plot(ECs, rtp_C, '.-', label="Coulomb")
    plt.legend()
    plt.show()  

In [43]:
plt.cla()
for i in range(0, nEs_keep):
    plt.plot(ls, run_data[:,i+1], 'o-')
plt.ylim(-25,5)
plt.show()

In [44]:
if safe_data:
    cols = ['z']
    for i in range(nEs_keep):
        cols.append('E'+str(i+1))
    df = pd.DataFrame(run_data, columns=cols)
    df.to_csv('rac_DVR_3D_coulomb_rmax_12.csv', index=False)
    df.head(5)

## RAC with soft-box

In [45]:
def softbox(r, rcut=1.0, lbd=1.0):
    """ 
    Softbox: 
    -1 at the origin, rises at r0 softly to asymptotic 0
    based on Gaussian with inverted scale
    """
    return lbd*(np.exp(-(2*rcut)**2/r**2) - 1)

def f(x, E, jparam, l_curr):
    return jolanta.Jolanta_3D(x, jparam) + softbox(x, rcut=3.0, lbd=l_curr) - E

In [46]:
#
#  show the box potential
#
plt.cla()
for l_curr in [0.1, 0.2, 0.3, 0.4, 0.5]:
    Vs = jolanta.Jolanta_3D(rs, jparam)
    Ws = softbox(rs, rcut=3.0, lbd=l_curr)
    plt.plot(rs, Ws*au2eV)

plt.xlim(0,20)
plt.ylim(-15, 0)
plt.show()

In [47]:
#
#  show the full potential
#
plt.cla()
for l_curr in [0.1, 0.2, 0.3, 0.4, 0.5]:
    Vs = jolanta.Jolanta_3D(rs, jparam)
    Ws = softbox(rs, rcut=3.0, lbd=l_curr)
    plt.plot(rs, (Vs+Ws)*au2eV)
    if turning_points:
        print(find_brackets(rs,Vs+Ws))

#plt.xlim(0,20)
plt.ylim(-30, 8)
plt.show()

[2.187845303867403, 8.552486187845304]
[2.187845303867403, 8.817679558011049]
[2.187845303867403, 9.082872928176796]
[2.187845303867403, 9.414364640883978]
[2.187845303867403, 9.679558011049723]


In [51]:
l_min=0.0
l_max=1.25
nEs_keep=4    # how many energies are kept

npts=101

ls=np.linspace(l_min, l_max, num=npts, endpoint=True)

rtp_SB=np.zeros(npts)


run_data = np.zeros((npts, nEs_keep+1))  # array used to collect all eta-run data
run_data[:,0]=ls

VJs = jolanta.Jolanta_3D(rs, jparam)
Ws = softbox(rs, rcut=3.0, lbd=1.0)

for j, l_curr in enumerate(ls):
    Vs = VJs + l_curr*Ws
    energy = dvr.DVRDiag2(nGrid, Ts, Vs)
    run_data[j,1:] = au2eV*energy[0:nEs_keep]
    if turning_points:
        args=(energy[1],jparam,l_curr)
        bracket = find_brackets(rs,Vs)
        sol=root_scalar(f, args=args, bracket=bracket)
        rtp_SB[j]=sol.root
    print(j+1, end=" ")
    if (j+1)%10==0:
        print()

if turning_points:
    ESBs = run_data[:,1]
        
print(run_data[-1,:])

1 2 3 4 5 6 7 8 9 10 
11 12 13 14 15 16 17 18 19 20 
21 22 23 24 25 26 27 28 29 30 
31 32 33 34 35 36 37 38 39 40 
41 42 43 44 45 46 47 48 49 50 
51 52 53 54 55 56 57 58 59 60 
61 62 63 64 65 66 67 68 69 70 
71 72 73 74 75 76 77 78 79 80 
81 82 83 84 85 86 87 88 89 90 
91 92 93 94 95 96 97 98 99 100 
101 [  1.25       -39.8962075  -24.06967741 -11.79294632  -4.15997774]


In [52]:
plt.cla()
for i in range(0, nEs_keep):
    plt.plot(ls, run_data[:,i+1], 'o-')
plt.ylim(-25,5)
plt.show()

In [53]:
if turning_points:
    plt.cla()
    plt.plot(Ebs, rtp_b, '.-', label="b-scaling")
    plt.plot(ECs, rtp_C, '.-', label="Coulomb")
    plt.plot(ESBs, rtp_SB, '.-', label="soft-box")
    plt.legend()
    plt.show() 

In [12]:
if safe_data:
    cols = ['z']
    for i in range(nEs_keep):
        cols.append('E'+str(i+1))
    df = pd.DataFrame(run_data, columns=cols)
    df.to_csv('rac_DVR_3D_softbox_rmax_12.csv', index=False)
    df.head(5)